# . Data Cleaning and Merging
## Overview
This notebook integrates our News Frequency data with three external indicators:
1.  **Military Power:** Sourced from Global Firepower (2017-2025).
2.  **GDP (Economic Power):** Sourced from the World Bank (Avg 2011-2020).
3.  **Democracy Index:** Sourced from the EIU.
4.  **Fragile States Index:** Sourced from Fragile States Index

## Key Challenges
* **Inconsistent Naming:** Handling variations like "Turkey" vs. "Türkiye" and "Ivory Coast" vs. "Côte d'Ivoire".
* **Missing Military Data:** Entities like **Hong Kong** and **Palestine** appear frequently in news but lack official military rankings.

### Processing GDP and Democracy Data

Here is the code that calculates the averages of GDP's(last 10 years) and Democracy(all available years) scores

In [ ]:
import pandas as pd
import numpy as np

# Defined set of regional/aggregate names to exclude
EXCLUDED_ENTITIES = {
    'World', 'Africa', 'Asia', 'Europe', 'Americas', 'North America', 
    'South America', 'Oceania', 'European Union', 'European Union (27)', 
    'Middle East and North Africa', 'High-income countries', 
    'Low-income countries', 'Lower-middle-income countries', 
    'Upper-middle-income countries', 'East Asia & Pacific (IDA & IBRD countries)', 
    'Latin America & Caribbean (IDA & IBRD countries)', 
    'Sub-Saharan Africa (IDA & IBRD countries)', 'OECD members', 
    'Non-OECD members', 'IDA countries', 'IBRD countries', 
    'Arab World', 'Central Europe and the Baltics', 'East Asia & Pacific', 
    'Europe & Central Asia', 'Euro area', 'Fragile and conflict affected situations', 
    'Heavily indebted poor countries (HIPC)', 'Least developed countries: UN classification',
    'Middle East & North Africa', 'North America', 'Other small states', 
    'Pacific island small states', 'Post-demographic dividend', 'Pre-demographic dividend',
    'Small states', 'South Asia', 'Sub-Saharan Africa', 
    'Upper middle income', 'Upper-middle-income countries', 'Least developed countries',
    'East Asia and Pacific (WB)', 'Europe and Central Asia (WB)', 
    'Latin America and Caribbean (WB)', 
    'Middle East, North Africa, Afghanistan and Pakistan (WB)', 
    'North America (WB)', 'South Asia (WB)', 'Sub-Saharan Africa (WB)'
}

# --- 1) Democracy Averages (Rounded to 2 decimal places) ---
df_democracy = pd.read_csv('democracy-index-eiu.csv')

# Filter out regions
df_countries_democracy = df_democracy[~df_democracy['Entity'].isin(EXCLUDED_ENTITIES)]

# Calculate the mean Democracy score
democracy_averages = df_countries_democracy.groupby('Entity')['Democracy score'].mean().reset_index()
democracy_averages.columns = ['Country', 'Average_Democracy_Score']

# Round the score to two digits after the dot
democracy_averages['Average_Democracy_Score'] = democracy_averages['Average_Democracy_Score'].round(2)

# Sort and save the result
democracy_averages = democracy_averages.sort_values(by='Average_Democracy_Score', ascending=False)
democracy_averages.to_csv('country_democracy_averages_rounded.csv', index=False)





# --- 2) GDP Average for Last 10 Available Years (Convert to Billions, round to 2 decimal places) ---
df_gdp = pd.read_csv('GDP_our_world_in_data.csv')

# Filter out regions
df_countries_gdp = df_gdp[~df_gdp['Entity'].isin(EXCLUDED_ENTITIES)]

# Define the function to get the GDP average for the last 10 years
def gdp_last_10_average(group):
    """Sorts data by year and returns the mean GDP of the last 10 available years."""
    # Sort by year in descending order
    group_sorted = group.sort_values(by='Year', ascending=False)
    # Take the top 10 available rows and calculate the mean of ny_gdp_mktp_kd
    return group_sorted.head(10)['ny_gdp_mktp_kd'].mean()

# Group by Entity and apply the custom function
gdp_averages = df_countries_gdp.groupby('Entity').apply(gdp_last_10_average).reset_index()
gdp_averages.columns = ['Country', 'Average_GDP_Last_10_Years_Raw']

# Convert the raw GDP value to Billions of USD (divide by 10^9) and round to 2 decimal places
gdp_averages['Average_GDP_Last_10_Years_Billion_USD'] = (
    gdp_averages['Average_GDP_Last_10_Years_Raw'] / 1e9
).round(2)

# Drop the raw value column
gdp_averages = gdp_averages.drop(columns=['Average_GDP_Last_10_Years_Raw'])

# Sort and save the result
gdp_averages = gdp_averages.sort_values(by='Average_GDP_Last_10_Years_Billion_USD', ascending=False)
gdp_averages.to_csv('country_gdp_last_10_averages_billions.csv', index=False)

C:\Users\mstf\AppData\Local\Temp\ipykernel_22096\2476430426.py:60: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  gdp_averages = df_countries_gdp.groupby('Entity').apply(gdp_last_10_average).reset_index()


Here, some mismatching countries between two datasets either inputed with guessed value or dropped

In [19]:
import pandas as pd
import numpy as np

# Load the previously calculated average files
df_democracy_avg = pd.read_csv('country_democracy_averages_rounded.csv')
df_gdp_avg = pd.read_csv('country_gdp_last_10_averages_billions.csv')

# --- 1. Imputation Data ---
# Note: These values are based on reasoned guessing using external context.

imputed_democracy_data = pd.DataFrame({
    'Country': ['Brunei', 'Kosovo', 'South Sudan', 'Somalia'],
    'Average_Democracy_Score': [2.00, 5.50, 1.50, 1.00]
})

imputed_gdp_data = pd.DataFrame({
    'Country': ['North Korea', 'Taiwan', 'Venezuela'],
    'Average_GDP_Last_10_Years_Billion_USD': [40.00, 700.00, 150.00]
})

# --- 2. Merge and Finalize Imputed Democracy Data ---
# Concatenate the new data and remove duplicates (keeping the imputed row where country names matched the previous missing list)
df_democracy_imputed = pd.concat([df_democracy_avg, imputed_democracy_data], ignore_index=True)

# Drop duplicates that may occur if the imputed countries existed in the raw data but were filtered out previously
df_democracy_imputed = df_democracy_imputed.drop_duplicates(subset=['Country'], keep='last')

# Sort the final list
df_democracy_imputed = df_democracy_imputed.sort_values(by='Average_Democracy_Score', ascending=False).reset_index(drop=True)

# Save the final imputed list
df_democracy_imputed.to_csv('country_democracy_averages_imputed.csv', index=False)


# --- 3. Merge and Finalize Imputed GDP Data ---
# Concatenate the new data
df_gdp_imputed = pd.concat([df_gdp_avg, imputed_gdp_data], ignore_index=True)

# Drop duplicates (same rationale as above)
df_gdp_imputed = df_gdp_imputed.drop_duplicates(subset=['Country'], keep='last')

# Sort the final list
df_gdp_imputed = df_gdp_imputed.sort_values(by='Average_GDP_Last_10_Years_Billion_USD', ascending=False).reset_index(drop=True)

# Save the final imputed list
df_gdp_imputed.to_csv('country_gdp_averages_imputed.csv', index=False)

print("\nImputation complete.")
print("The final Democracy Averages (Imputed) are saved to: country_democracy_averages_imputed.csv")
print("The final GDP Averages (Imputed) are saved to: country_gdp_averages_imputed.csv")


Imputation complete.
The final Democracy Averages (Imputed) are saved to: country_democracy_averages_imputed.csv
The final GDP Averages (Imputed) are saved to: country_gdp_averages_imputed.csv


### Processing Military Data
We create a **"PowerIndex"** by averaging historical rankings.
* **Metric:** PwrIndx (0.00 is perfect score).
* **Inversion:** For analysis, we will later invert this score so that *higher values = stronger military*.

In [3]:
import pandas as pd
import glob

def process_military_data():
    # Get all military csv files
    files = glob.glob('*_military.csv')
    
    dfs = []
    
    # Standardize column names across different years
    col_map = {
        'name': 'Country',
        'COUNTRY': 'Country',
        'country': 'Country',
        '2020 ranking': 'Country',
        'power_index': 'PowerIndex',
        'POWER INDEX': 'PowerIndex',
        'pwr_index': 'PowerIndex',
        'Unnamed: 1': 'PowerIndex'
    }
    
    for file in files:
        try:
            df = pd.read_csv(file)
            
            # Normalize column names
            df = df.rename(columns=col_map)
            
            # Check for required columns
            if 'Country' in df.columns and 'PowerIndex' in df.columns:
                subset = df[['Country', 'PowerIndex']].copy()
                
                # Convert to numeric, handle errors
                subset['PowerIndex'] = pd.to_numeric(subset['PowerIndex'], errors='coerce')
                subset = subset.dropna(subset=['PowerIndex'])
                
                # Clean country names
                subset['Country'] = subset['Country'].astype(str).str.strip()
                
                # --- FIX 1: Combine "Turkey" and "Türkiye" ---
                subset['Country'] = subset['Country'].replace({
                    'Turkey': 'Türkiye',
                    'Turkiye': 'Türkiye'
                })
                
                dfs.append(subset)
                
        except Exception as e:
            print(f"Error processing {file}: {e}")
    
    if dfs:
        # Combine all data
        combined_df = pd.concat(dfs, ignore_index=True)
        
        # Calculate Average
        result = combined_df.groupby('Country')['PowerIndex'].mean().reset_index()
        
        # --- FIX 2: Round to 2 decimal places ---
        result['PowerIndex'] = result['PowerIndex'].round(2)
        
        # Save to CSV
        output_filename = 'country_average_power_index.csv'
        result.to_csv(output_filename, index=False)
        
        print(f"Successfully created '{output_filename}'")
        print(result[result['Country'] == 'Türkiye'])  # Verify the fix
        print(result.head())
    else:
        print("No data found.")

if __name__ == "__main__":
    process_military_data()

Successfully created 'country_average_power_index.csv'
     Country  PowerIndex
134  Türkiye        0.21
       Country  PowerIndex
0  Afghanistan        2.21
1      Albania        2.16
2      Algeria        0.43
3       Angola        0.93
4    Argentina        0.56


### Processing Fragile States Index Data


In [1]:
import pandas as pd
import numpy as np
import glob
import re
import os

# --- Configuration ---
COUNTRY_MAPPING = {
    'Brunei Darussalam': 'Brunei', 'Cabo Verde': 'Cape Verde', 
    'Congo Democratic Republic': 'DR Congo', 'Democratic Republic of Congo': 'DR Congo',
    'Congo Republic': 'Congo', 'Congo': 'Congo', 'Czech Republic': 'Czechia',
    'Kyrgyz Republic': 'Kyrgyzstan', 'Macedonia': 'North Macedonia', 
    'Slovak Republic': 'Slovakia', 'Timor-Leste': 'East Timor',
    'Micronesia': 'Micronesia (country)', 'Türkiye': 'Turkey',
    'Israel and West Bank': 'Israel' 
}

EXCLUDED_ENTITIES = {
    'Country', 'World', 'Africa', 'Asia', 'Europe', 'North America', 
    'South America', 'Oceania', 'Rank'
}

# --- Processing ---
fsi_files = glob.glob('Rankings*.csv')
all_fsi_data = []

print(f"Found {len(fsi_files)} files. Processing...")

for file_path in fsi_files:
    try:
        # Read the file with the expected separator and header row (index 1 for FSI files)
        # Handle encoding: try utf-8 first, fallback to latin-1
        try:
             df = pd.read_csv(file_path, sep='\t', header=1, encoding='utf-16')
        except UnicodeDecodeError:
             df = pd.read_csv(file_path, sep='\t', header=1, encoding='latin-1')
        
        # Identify the Year/Total Score column
        year_col = None
        # Strategy 1: Look for a column named exactly 'Total'
        if 'Total' in df.columns:
             year_col = 'Total'
        else:
            # Strategy 2: Look for a column that is a 4-digit year (e.g., '2020')
            for col in df.columns:
                if re.match(r'^\d{4}$', str(col)):
                    year_col = col
                    break
        
        if year_col:
            # Select and rename
            df_clean = df[['Country', year_col]].copy()
            df_clean = df_clean.rename(columns={year_col: 'Total_Score'})
            
            # Basic cleaning
            df_clean = df_clean.dropna(subset=['Country'])
            df_clean['Country'] = df_clean['Country'].replace(COUNTRY_MAPPING)
            
            # Ensure Total_Score is numeric
            df_clean['Total_Score'] = pd.to_numeric(df_clean['Total_Score'], errors='coerce')
            
            all_fsi_data.append(df_clean)
            print(f"  - Processed: {file_path} (Using column: '{year_col}')")
        else:
            print(f"  - WARNING: Could not find 'Total' or Year column in {file_path}")

    except Exception as e:
        print(f"  - ERROR processing {file_path}: {e}")

# --- Aggregation and Saving ---
if all_fsi_data:
    df_combined = pd.concat(all_fsi_data, ignore_index=True)
    
    # Calculate Mean of Total Score for each country
    df_averages = df_combined.groupby('Country')['Total_Score'].mean().round(2).reset_index()
    df_averages.columns = ['Country', 'Average_FSI_Total_Score']
    
    # Filter Exclusions
    df_averages = df_averages[~df_averages['Country'].isin(EXCLUDED_ENTITIES)]
    
    # Sort
    df_averages = df_averages.sort_values(by='Average_FSI_Total_Score', ascending=False)
    
    # Save to CSV
    output_filename = 'country_average_fsi_score.csv'
    df_averages.to_csv(output_filename, index=False)
    
    print(f"\n✅ SUCCESS: '{output_filename}' has been created.")
    print("Top 5 Fragile States (Average):")
    print(df_averages.head())
else:
    print("\n❌ FAILED: No data could be extracted.")

Found 10 files. Processing...
  - Processed: Rankings_2015.csv (Using column: '2015')
  - Processed: Rankings_2016.csv (Using column: '2016')
  - Processed: Rankings_2017.csv (Using column: '2017')
  - Processed: Rankings_2018.csv (Using column: '2018')
  - Processed: Rankings_2019.csv (Using column: '2019')
  - Processed: Rankings_2020.csv (Using column: '2020')
  - Processed: Rankings_2021.csv (Using column: '2021')
  - Processed: Rankings_2022.csv (Using column: '2022')
  - Processed: Rankings_2023.csv (Using column: '2023')
  - Processed: Rankings_2024.csv (Using column: '2024')

✅ SUCCESS: 'country_average_fsi_score.csv' has been created.
Top 5 Fragile States (Average):
         Country  Average_FSI_Total_Score
149      Somalia                   112.24
152  South Sudan                   111.39
177        Yemen                   110.83
159        Syria                   109.71
45      DR Congo                   108.96


#### Combining Datasets

In [ ]:
import pandas as pd

# 1. Load All 5 Datasets (Added FSI)
counts_df = pd.read_csv('filtered_country_counts.csv')
gdp_df = pd.read_csv('country_gdp_averages_imputed.csv')
demo_df = pd.read_csv('country_democracy_averages_imputed.csv')
power_df = pd.read_csv('country_average_power_index.csv')
fsi_df = pd.read_csv('country_average_fsi_score.csv')  # <--- NEW FILE

# 2. Clean GDP, Democracy, and FSI Data (Apply Name Corrections)
name_corrections = {
    "Turkey": "Türkiye",
    "Russian Federation": "Russia",
    "Egypt, Arab Rep.": "Egypt",
    "Iran, Islamic Rep.": "Iran",
    "Syrian Arab Republic": "Syria",
    "Venezuela, RB": "Venezuela",
    "Yemen, Rep.": "Yemen",
    "Korea, Rep.": "South Korea",
    "Korea, Dem. People's Rep.": "North Korea",
    "Slovak Republic": "Slovakia",
    "Czech Republic": "Czechia",
    "Kyrgyz Republic": "Kyrgyzstan",
    "Lao PDR": "Laos",
    "Congo, Dem. Rep.": "DR Congo",
    "Congo, Rep.": "Congo",
    "Gambia, The": "Gambia",
    "Bahamas, The": "Bahamas",
    "Brunei Darussalam": "Brunei",
    "Cabo Verde": "Cape Verde",
    "Timor-Leste": "East Timor",
    "United States": "United States"
}

# Apply corrections to GDP and Democracy
gdp_df['Country'] = gdp_df['Country'].replace(name_corrections)
demo_df['Country'] = demo_df['Country'].replace(name_corrections)
demo_df.rename(columns={'Entity': 'Country'}, inplace=True)

# Apply corrections to FSI (Specifically Turkey -> Türkiye)
fsi_df['Country'] = fsi_df['Country'].replace(name_corrections)

# 3. Clean Military Power Data
power_df.dropna(subset=['PowerIndex'], inplace=True)
power_df['PowerIndex'] = pd.to_numeric(power_df['PowerIndex'], errors='coerce')

# Fix Military Name Mismatches
military_name_map = {
    'Ivory Coast': "Côte d'Ivoire",
    'Democratic Republic of the Congo': 'DR Congo', 
    'Republic of the Congo': 'Congo',           
    'Beliz': 'Belize',
    'Turkey': 'Türkiye' 
}
power_df['Country'] = power_df['Country'].replace(military_name_map)
power_df = power_df.groupby('Country')['PowerIndex'].mean().reset_index()


# 4. Merge All 5 Datasets Directly
# Chain the merges: Frequency -> GDP -> Democracy -> Military -> FSI
final_complete_df = counts_df.merge(gdp_df, on='Country', how='inner') \
                             .merge(demo_df, on='Country', how='inner') \
                             .merge(power_df, on='Country', how='inner') \
                             .merge(fsi_df, on='Country', how='left') # Left join for FSI to keep main countries

# 5. Impute Missing FSI Scores
# (Taiwan, HK, Kosovo are usually missing from FSI but present in others)
fsi_imputations = {
    'Taiwan': 33.50,
    'Hong Kong': 35.00,
    'Kosovo': 68.00
}

for country, score in fsi_imputations.items():
    mask = (final_complete_df['Country'] == country) & (final_complete_df['Average_FSI_Total_Score'].isna())
    if mask.any():
        final_complete_df.loc[mask, 'Average_FSI_Total_Score'] = score

# 6. Save the Final Result
output_file = 'final_complete_dataset.csv'
final_complete_df.to_csv(output_file, index=False)

print("Merge Complete!")
print(final_complete_df.head())